# Set the variables below

In [1]:
GTFS_FILE = "./data/BUCHAREST.zip"
OSM_FILE = "./data/bucuresti.json"

# CSV format (headerless): gtfs_stop_id,gtfs_stop_name,osm_item_id,osm_stop_name
OUTPUT_FILE = "correlate_bu.csv"

FILTER_ALREADY_CORRELATED_DATA = True


In [2]:
import csv
import logging

from ipyleaflet import Map, GeoData, Popup, projections, FullScreenControl
from ipywidgets import HTML
from ipywidgets import HBox, VBox, Text, Layout

from gtfs_functions import Feed


logging.basicConfig(level=logging.INFO)
log = logging.getLogger()

import utils

state = utils.State()
swap = lambda x: (x[1], x[0])


def on_gtfs_click(properties, **kw):
    global state, status_bar

    # properties = {'stop_id': '6001', 'stop_code': None, 'stop_name': 'PITT', 'stop_desc': None, 'stop_lat': 45.795162, 'stop_lon': 21.175718, 'zone_id': None, 'stop_url': None, 'location_type': 0, 'parent_station': None, 'stop_timezone': None, 'wheelchair_boarding': 0, 'platform_code': None},

    # use from_kwargs because we don't map all attributes in GtfsStop and it raises
    # an exception on unknown ones
    state.last_clicked_gtfs_element = utils.GtfsStop.from_kwargs(**properties)

    status_bar.value = f"selected GTFS node: {state.last_clicked_gtfs_element.stop_name}"


    log.info(f"clicked GTFS element: {state.last_clicked_gtfs_element}")

    if state.both_nodes_set():
        utils.write_correlation_row(state, OUTPUT_FILE)
        status_bar.value = f"wrote CSV row: {state.last_clicked_gtfs_element.stop_name} <-> {state.last_clicked_osm_element.stop_name}"
        state.reset()


def on_osm_click(properties, **kw):
    global state, status_bar

    # use from_kwargs because we don't map all attributes in OsmStop and it raises
    # an exception on unknown ones
    state.last_clicked_osm_element = utils.OsmStop.from_kwargs(**properties)


    status_bar.value = f"selected OSM node: {state.last_clicked_osm_element.stop_name}"


    log.info(f"clicked OSM element: {state.last_clicked_osm_element}")

    # TODO: find a way for removing the clicked nodes after correlating them
    
    # for layer in m.layers:
    #     log.info(f"---> {layer}")

    if state.both_nodes_set():
        utils.write_correlation_row(state, OUTPUT_FILE)
        status_bar.value = f"wrote CSV row: {state.last_clicked_gtfs_element.stop_name} <-> {state.last_clicked_osm_element.stop_name}"
        state.reset()


# Define the on_hover event handler
def on_osm_hover(event, feature, **kwargs):
    # Get the properties of the hovered feature
    properties = feature["properties"]

    coords = feature["geometry"]["coordinates"]

    # Create a popup with the feature's information
    popup_content = HTML()
    popup_content.value = f"<b>OSM</b> \
    (stop id: <a href=\"https://www.openstreetmap.org/node/{properties['stop_id']}\" target=\"_blank\">{properties['stop_id']}</a>) \
    <br>{properties['stop_name']}<br>" \
    f"({coords[1]}, {coords[0]})"

    # Create a popup object and attach it to the map
    popup = Popup(
        auto_pan=False,
        location=swap(feature["geometry"]["coordinates"]),
        child=popup_content,
        close_button=False,
    )
    m.add_layer(popup)


def on_gtfs_hover(event, feature, **kwargs):
    properties = feature["properties"]

    log.info(f"Feature={feature}")

# {'id': '193', 'type': 'Feature', 'properties': {'stop_id': '12026', 'stop_code': None, 'stop_name': 'Biserica Penticostala', 
# 'stop_desc': None, 'stop_lat': 0, 'stop_lon': 0, 'zone_id': None, 'stop_url': None, 'location_type': 0, 'parent_station': None, 
# 'stop_timezone': None, 'wheelchair_boarding': 0, 'platform_code': None}, 'geometry': {'type': 'Point', 'coordinates': [0, 0]}}    

    # Create a popup with the feature's information
    popup_content = HTML()
    popup_content.value = f"<b>GTFS</b> (stop id: {properties['stop_id']})<br>{properties['stop_name']}<br>" \
                        f"({properties['stop_lat']}, {properties['stop_lon']})"

    # Create a popup object and attach it to the map
    popup = Popup(
        auto_pan=False,
        location=swap(feature["geometry"]["coordinates"]),
        child=popup_content,
        close_button=False,
    )
    m.add_layer(popup)


# Open the GTFS file
f = Feed(GTFS_FILE)


gtfs_nodes = f.stops
osm_nodes = utils.osm_2_gdf(OSM_FILE)

if FILTER_ALREADY_CORRELATED_DATA:
    try:
        gtfs_nodes, osm_nodes = utils.filter_correlated_data(OUTPUT_FILE, gtfs_nodes, osm_nodes)
    except IOError:
        # ignore file not found
        pass 
    except Exception as e:
        log.error(f"error filtering already correlated data: {e}")

# Create the map

center = (45.7600770, 21.2604270)
m = Map() #center=center)  # 
# crs=projections.EPSG4326) --> this screws things up, not sure why

# add the button for making the whole thing fullscreen
m.add_control(FullScreenControl())


# TODO: a status bar or something similar inside the map

gd = GeoData(
    geo_dataframe=gtfs_nodes,
    style={
        "color": "blue",
        "radius": 5,
        "fillColor": "#3366cc",
        "opacity": 0.5,
        "weight": 1,
        "fillOpacity": 0.6,
    },
    # hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
    point_style={
        "radius": 3,
        "color": "red",
        "fillOpacity": 0.8,
        "fillColor": "blue",
        "weight": 3,
    },
)

gd.on_click(on_gtfs_click)
gd.on_hover(on_gtfs_hover)


osm_gd = GeoData(
    geo_dataframe=osm_nodes,
    style={
        "color": "red",
        "radius": 5,
        "fillColor": "#ff0000",
        "opacity": 0.5,
        "weight": 1,
        "fillOpacity": 0.6,
    },
    point_style={
        "radius": 3,
        "color": "red",
        "fillOpacity": 0.8,
        "fillColor": "red",
        "weight": 3,
    },
)

osm_gd.on_click(on_osm_click)
osm_gd.on_hover(on_osm_hover)


m.add_layer(gd)
m.add_layer(osm_gd)

layout = Layout(width="600px")
# Create a status box
status_bar = Text(
    value="",
    description="Status: ",
    disabled=True,
    layout=layout,
)


hbox = HBox([status_bar])

# Create the horizontal container containing the map and the horizontal container
vbox = VBox([m, hbox])
vbox

INFO:root:Reading "stops.txt".


FileNotFoundError: [Errno 2] No such file or directory: 'correlate_bu.csv'